In [1]:
# 데이터를 쉽게 다루기 위해 Pandas 라이브러리를 불러옵니다.
import pandas as pd

# CSV 파일을 불러옵니다.
data = pd.read_csv('data/temperature_electricity.csv', encoding='UTF8')

# 숫자가 아닌 데이터를 삭제합니다.
data = data.dropna()

In [2]:
# 데이터가 어떤 구성으로 되어 있는지 확인합니다.
data

,Date,Location,Average,Lowest,Highest,Capacity(ins),Capacity(Power),Max Power,Power Reserve,Power Reserve Ratio(%),Max Power Time
0,2013-06-13,108,23.0,18.8,27.5,83749,71175,66272,4903,7.4,2013/06/13(15:00)
1,2013-06-14,108,24.6,20.9,29.7,83749,71209,66475,4734,7.1,2013/06/14(15:00)
2,2013-06-15,108,24.6,21.3,29.1,83749,67663,60427,7236,12.0,2013/06/15(15:00)
3,2013-06-16,108,22.5,19.8,27.5,83749,66461,53717,12744,23.7,2013/06/16(21:00)
4,2013-06-17,108,22.3,19.9,25.8,83749,70892,66566,4326,6.5,2013/06/17(15:00)
...,...,...,...,...,...,...,...,...,...,...,...
3648,2023-06-09,108,23.3,18.3,28.4,140758,83619,69747,13872,19.9,2023/06/09(19:00)
3649,2023-06-10,108,23.2,19.6,28.5,140758,82997,61953,21044,34.0,2023/06/10(21:00)
3650,2023-06-11,108,22.1,17.9,27.3,140758,82473,62339,20134,32.3,2023/06/11(21:00)
3651,2023-06-12,108,22.2,18.4,27.5,140758,87631,71512,16119,22.5,2023/06/12(18:00)


In [3]:
# 평균 기온, 최고 기온, 최저 기온 데이터를 불러옵니다.
temperature =data.loc[:, ["Average", "Highest", "Lowest"]].apply(pd.to_numeric)

# 전력 예비율 데이터를 불러옵니다.
electricity_data = data["Power Reserve Ratio(%)"]

In [4]:
# 전력 예비율 데이터를 10 크기의 구간 별로 나누어줍니다.
electricity = {"class":[]}

for i in electricity_data:
    if i < 10:
        electricity["class"].append(0)
    elif i < 20:
        electricity["class"].append(1)
    elif i < 30:
        electricity["class"].append(2)
    elif i < 40:
        electricity["class"].append(3)
    elif i < 50:
        electricity["class"].append(4)
    elif i < 60:
        electricity["class"].append(5)
    elif i < 70:
        electricity["class"].append(6)
    elif i < 80:
        electricity["class"].append(7)
    elif i < 90:
        electricity["class"].append(8)
    else:
        electricity["class"].append(9)

# Dictionary 형태의 데이터를 DataFrame 형태로 변환해줍니다.
electricity = pd.DataFrame(electricity)

In [5]:
# 학습할 데이터의 형태를 확인합니다.
print(temperature.shape, electricity.shape)

(3651, 3) (3651, 1)


In [7]:
# 데이터를 훈력 데이터와 테스트 데이터로 나누어줍니다.
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(temperature, electricity, test_size=0.3)
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [8]:
# 학습할 데이터의 형태를 확인합니다.
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(2555, 3) (2555,)
(1096, 3) (1096,)


In [9]:
# SciKit-Learn 라이브러리에서 Gaussian Naive Bayes 모델을 불러옵니다.
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

# 모델을 학습시킵니다.
model.fit(X_train, y_train)

# 정확도를 측정합니다.
y_pred = model.predict(X_test)

In [10]:
# Scikit-learn 라이브러리의 metrics를 사용하여 정확도를 계산합니다.
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.4041970802919708


In [11]:
# 최고 정확도를 저장할 변수를 선언합니다.
higest_accuracy = 0

# 1000번 반복하여 가장 높은 정확도를 구합니다.
for i in range(1000):
    data = pd.read_csv('data/temperature_electricity.csv', encoding='UTF8')
    data = data.dropna()
    temperature =data.loc[:, ["Average", "Highest", "Lowest"]].apply(pd.to_numeric)
    electricity_data = data["Power Reserve Ratio(%)"]
    electricity = {"class":[]}

    for j in electricity_data:
        if j < 10:
            electricity["class"].append(0)
        elif j < 20:
            electricity["class"].append(1)
        else:
            electricity["class"].append(2)

    electricity = pd.DataFrame(electricity)
    X_train, X_test, y_train, y_test = train_test_split(temperature, electricity, test_size=0.3)
    y_train = y_train.values.ravel()
    y_test = y_test.values.ravel()
    model = GaussianNB()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    if higest_accuracy < metrics.accuracy_score(y_test, y_pred):
        higest_accuracy = metrics.accuracy_score(y_test, y_pred)

In [12]:
# 최고 정확도를 출력합니다.
print("Higest Accuracy :", higest_accuracy*100, "%")

Higest Accuracy : 73.44890510948905 %


In [13]:
# 학습한 알고리즘을 사용하여 임의의 기온 데이터에 따른 전력 예비율을 예측합니다.
predict_data = {"Average":[25], "Highest":[30], "Lowest":[23]}
predict_data = pd.DataFrame(predict_data)
print("Class", model.predict(predict_data)[0])
# Class 2로 예측하여 평균 25도, 최고 30도, 최저 23도일 때 전력 예비율이 10%에서 20% 사이에 있을 것임을 예측하였습니다.

Class 2
